Recap: Computing aggregates over large relations

```sql
SELECT max(salary), min(salary), avg(salary)
FROM employee, department
WHERE dno = dnumber and dnae = 'Research'
```

1. compute the relation dictated by FROM and WHERE clauses
2. for every such client, extract all the salaries
3. from that collection, apply the 3 aggregate functions 

Recall the Company Schema from prior lectures:

![](images/2022-10-20-00-05-41.png)

## Grouping

Allows aggregates to be applied to *subgroups of tuples in a relation*
- function applied to each subgroup independently

**Subgroup**: set of tuples that have the *same value* for the *grouping attribute(s)*

SQL syntax: GROUP BY clause specifies the grouping attribute (*which must also appear in the SELECT clause*)



aggregate ALL employees vs aggregate for each department:

```sql
SELECT avg(Salary) AS AvgSal
FROM employee
```

```sql
SELECT Dept, avg(Salary) AS AvgSal
FROM Employee
GROUP BY Dept
```

![](images/2022-10-20-00-08-46.png)

For each department, retrieve
- department number
- number of employees in the department
- average salary

```sql
SELECT DNO, COUNT(*) AS NUMEMP, AVG(SALARY) AS AVGSAL
FROM EMPLOYEE
GROUP BY DNO
```

for each project, retrieve
- project number
- project name
- number of employees who work on that project

```sql
SELECT      PNUMBER, PNAME, COUNT(*)
FROM        PROJECT, WORKS_ON
WHERE       PNUMBER=PNO
GROUP BY    PNUMBER, PNAME
```

- FROM-WHERE combo essentially combines the PROJECT & WORKS_ON relations via the PNUMBER/PNO values. Tuples that share those values are concatenated into a single tuple to create a new unnamed intermediate relation w/ these shared values
- GROUP BY can be done on multiple attributes (more selective/specific groups)


> Subtlety: suppose PNO and ESSN do not form a key for WORKS_ON (e.g. logging hours spent on project)

Problem: will get duplicate employees

![](images/2022-10-20-00-44-36.png)

Fix:
```sql
SELECT      PNUMBER, PNAME, COUNT(DISTINCT ESSN)
FROM        PROJECT, WORKS_ON
WHERE       PNUMBER=PNO
GROUP BY    PNUMBER, PNAME
```

## HAVING Clause

Use case: retrieve values of aggregate functions for only those groups that satisfy certain conditions

specifies selection condition on GROUPS (as opposed to INDIVIDUAL TUPLES)

EX: Find the names of all branches where the average account balance is more than $1,200

```sql
SELECT      branch_name, avg(balance)
FROM        account
GROUP BY    branch_name
HAVING      avg(balance) > 1200
```

WHERE
- individual tuples themselves
- conditionals on attributes
- before groups are formed

HAVING
- on the totality of tuples within groups
- conditionals on 
  - group-by attribute values
  - aggregates(other attributes)
- after groups are formed

EX: for each proj w/ 2+ employees, retrieve
- proj number
- proj name
- num employees working on pro

```sql
SELECT      pnumber, pname, count(*)
FROM        project, works_on
WHERE       pnumber = pno
GROUP BY    pnumber, pname
HAVING      count(*) > 2
```

EX: For each movie having more than 100 actors (i.e. large cast), find the number of theatres showing the movie

```sql
SELECT      m.Title, count(distinc s.Theater) as number -- 4. Account for same theatre showing movie multiple times
FROM        Schedule s, Movie m                         -- 1. form an intermediate table by linking Schedule and Movie relations
WHERE       s.Title = m.Title                           --    via their Titles (concatenate each tuple sharing same movie title)
GROUP BY    m.Title                                     -- 2. Group by Titles in order to do the next check on movies:
HAVING      count(distinct m.Actor)>100                 -- 3. Filter for large casts (account for same actor in multiple roles)
```

> Note: Aggregate is taken over pairs <s,m> w/ same Title

## SQL Queries: Nesting

WHERE clause can contain predicates of the form

`attr/value IN <SQL query>`
`attr/value NO IN <SQL query>`

Semantics: IN predicate is satisfied if the attr/value appears in the result of the nested <SQL query>

EX: Find directors of current movies

```sql
SELECT  director FROM Movie
WHERE   title IN (
                    SELECT title
                    FROM schedule
                )
```

> Nested query returns the collection of titles of currently playing movies. From which the outer WHERE clause performs a membership conditional check on the title against the nested query.


EX: Find actors playing in some movie by Bertolucci

```sql
SELECT  Actor
FROM    Movie
WHERE   title IN (
                    SELECT  title
                    FROM    Movie
                    WHERE   director = "Bertolucci"
                )
```

> Nested query returns the collection of titles of movies by Bertolucci. From which the outer WHERE clause performs a membership conditional check on the title against the nested query.

#### On Nesting...

Key principle behind the design of the SQL Query Language => compositionality

The ability to nest SQL Queries is an important aspect of SQL - i.e. piping together smaller simpler expressions to make increasingly complicated expressions.

SQL designers/engineers did to make this feature happen?

`SELECT-FROM-WHERE` blocks input types and output types are the same: relations/tables. This makes it pluggable!

#### In terms of performance

Ideal: optimizers realize 2 diff expressions that are syntactically diff but **semantically equivalent** should optimize both in the same way

Realistically: optimization is computationally difficult. Usually nesting is SLOWER.